# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


In [43]:
import pandas as pd 
import numpy as np

In [44]:
recipes = pd.read_csv('recipes.tsv', sep='\t')
recipes

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...
...,...,...,...
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...


In [45]:
fields = []
for col in ["recipe_title", "recipe_introduction", "recipe_slug"]:
    if col in recipes.columns:
        fields.append(recipes[col].astype(str).str.lower().fillna(""))
text = fields[0]
for f in fields[1:]:
    text = (text + " " + f).fillna("")

In [46]:
# Remove pork products as I dont eat pork
pork_words = [
    "pork","bacon","ham","pepperoni"
]
# items I do enjoy
good_foods = [
    "chocolate","brownie","cake","cookie","dessert","sweet", "falafel","shawarma","hummus","chicken"
]

In [47]:
def has_any(s, words):
    return any(w in s for w in words)

def assign_rating(s):
    s = s or ""
    if has_any(s, pork_words):
        return 0.0
    if has_any(s, good_foods):
        return 1.0
    return 0.5

ratings = text.apply(assign_rating)

# convert to 1–5 raw scale
rating_raw = (1 + 4 * ratings).round().astype(int)


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [72]:
ratings_df = pd.DataFrame({
    "recipe_slug": recipes["recipe_slug"],
    "rating": (rating_raw - 1) * 0.25
})

In [73]:
ratings_df.to_csv("ratings.tsv", sep="\t", index=False)

In [74]:
ratings_df

,recipe_slug,rating
0,falafel,1.0
1,spamburger,0.0
2,bacon-fried-rice,0.0
3,chicken-fingers,1.0
4,apple-crisp,1.0
...,...,...
95,bacon-mac-and-cheese,0.0
96,chicken-alfredo-lasagna,1.0
97,classic-beef-lasagna,0.5
98,vegetarian-mushroom-lasagna,0.5


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [75]:
# YOUR CHANGES HERE

ratings = pd.read_csv("ratings.tsv", sep="\t")
tags = pd.read_csv("recipe-tags.tsv", sep="\t") 

In [76]:
tags

,recipe_slug,recipe_tag
0,spam-musubi,hawaiian
1,spam-musubi,nori
2,spam-musubi,onthego
3,spam-musubi,rice
4,spam-musubi,snack
...,...,...
747,bacon-souffle,breakfast
748,bacon-souffle,brunch
749,bacon-souffle,cheese
750,bacon-souffle,eggs


In [77]:
onehot = pd.crosstab(tags["recipe_slug"], tags["recipe_tag"])

# get features
features = ratings[["recipe_slug"]].merge(
    onehot, how="left", left_on="recipe_slug", right_index=True
).fillna(0).astype({c:int for c in onehot.columns})

# add bias columns
features.insert(1, "bias", 1)

features.to_csv("features.tsv", sep="\t", index=False)


/tmp/ipykernel_1951/2080559494.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features.insert(1, "bias", 1)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [78]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge

In [79]:
features = pd.read_csv("features.tsv", sep="\t")
ratings = pd.read_csv("ratings.tsv", sep="\t")

In [80]:
X = features.drop(columns=["recipe_slug"])
y = ratings["rating"]

model = Ridge(alpha=1.0, fit_intercept=False)
model.fit(X, y)

coef_df = pd.DataFrame({
    "recipe_tag": X.columns,
    "coefficient": model.coef_
})

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [81]:
# YOUR CHANGES HERE

coef_df.to_csv("model.tsv", sep="\t", index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [82]:
# YOUR CHANGES HERE

features = pd.features = pd.read_csv("features.tsv", sep="\t")
coef_df  = pd.read_csv("model.tsv", sep="\t")


In [83]:
X = features.drop(columns=["recipe_slug"])
coefs = coef_df.set_index("recipe_tag")["coefficient"]

In [84]:
for c in X.columns:
    if c not in coefs.index:
        coefs.loc[c] = 0.0
coefs = coefs.reindex(X.columns).fillna(0.0)

In [85]:
scores = X.values @ coefs.values

In [86]:
estimates = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_estimate": scores
})
estimates.to_csv("estimates.tsv", sep="\t", index=False)

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [91]:
# YOUR CHANGES HERE

ridge_reg = 1
ucb_alpha = 2

features = pd.read_csv("features.tsv", sep="\t")
ratings  = pd.read_csv("ratings.tsv",  sep="\t")

In [92]:
X = features.drop(columns=["recipe_slug"]).to_numpy(dtype=float)
y = ratings["rating"].to_numpy(dtype=float)

A = ridge_reg * np.eye(X.shape[1]) + X.T @ X
b = X.T @ y
theta = np.linalg.solve(A, b)

In [93]:
bounds = []
for i, slug in enumerate(features["recipe_slug"]):
    x = X[i, :]
    z = np.linalg.solve(A, x)
    var_term = float(np.sqrt(x @ z))
    estimate = float(x @ theta)
    ucb = estimate + ucb_alpha * var_term
    bounds.append((slug, ucb))

In [94]:
pd.DataFrame(bounds, columns=["recipe_slug", "score_bound"]).to_csv("bounds.tsv", sep="\t", index=False)

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [67]:
# YOUR CHANGES HERE

X_df = pd.read_csv("features.tsv", sep="\t")
y_df = pd.read_csv("ratings.tsv",  sep="\t")

In [68]:

X = X_df.drop(columns=["recipe_slug"]).to_numpy(float)
y = y_df["rating"].to_numpy(float)
slugs = X_df["recipe_slug"].tolist()

In [69]:
lam = 1.0
alpha = 2.0
rounds = 100

d = X.shape[1]
A = lam * np.eye(d)
b = np.zeros(d)

available = set(range(len(slugs)))
results = []

In [70]:
for _ in range(min(rounds, len(slugs))):
    theta = np.linalg.solve(A, b)

    # score LinUCB bound for each available recipe
    best_i, best_bound = None, -np.inf
    for i in available:
        x = X[i]
        z = np.linalg.solve(A, x)
        ucb = float(x @ theta) + alpha * float(np.sqrt(x @ z))
        if ucb > best_bound:
            best_i, best_bound = i, ucb

    r = float(y[best_i])
    x = X[best_i]

    # update
    A += np.outer(x, x)
    b += r * x

    results.append({"recipe_slug": slugs[best_i], "score_bound": round(best_bound, 6), "reward": r})
    available.remove(best_i)

Submit "recommendations.tsv" in Gradescope.

In [71]:
pd.DataFrame(results).to_csv("recommendations.tsv", sep="\t", index=False)

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.